In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import joblib

# 데이터 로드
data = pd.read_csv('Data/HR_Employee.csv')

# 1. 타겟 변수와 입력 데이터 분리
y = data['Attrition']
X = data.drop(columns=['Attrition'])

# 2. 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 3. 타겟 변수 라벨 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 4. 이상치 처리 - MonthlyIncome 로그 변환
if 'MonthlyIncome' in data.columns:
    X['MonthlyIncome_Log'] = np.log1p(data['MonthlyIncome'])
    X.drop(columns=['MonthlyIncome'], inplace=True)

# 5. 파생변수 생성
X['YearsWithOtherCompanies'] = (X['TotalWorkingYears'] - X['YearsAtCompany']).apply(lambda x: max(x, 0))
X['AgeAtJoining'] = (X['Age'] - X['YearsAtCompany']).apply(lambda x: max(x, 0))
X['IncomePerYearWorked'] = X['MonthlyIncome_Log'] / (X['TotalWorkingYears'] + 1)

# 6. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. SMOTE 적용 (훈련 데이터에만)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 8. 누락된 열 자동 추가 함수 정의
def add_missing_columns(df, required_columns):
    missing_cols = [col for col in required_columns if col not in df.columns]
    for col in missing_cols:
        df[col] = 0
    return df[required_columns]

# 9. 스케일링 및 전처리 파이프라인 정의
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# ColumnTransformer에 수치형 처리 추가
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_features)
])

# 전처리 및 모델 학습 파이프라인
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=100,  penalty='l2', solver='lbfgs', C=0.5, random_state=42))
])

# 10. 모델 학습
model_pipeline.fit(X_train_resampled, y_train_resampled)

# 11. 테스트 데이터 변환 및 정확도 확인
X_test_aligned = add_missing_columns(X_test, X_train.columns)  # 누락된 열 추가
accuracy = model_pipeline.score(X_test_aligned, y_test)
print(f"테스트 데이터 정확도: {accuracy:.4f}")

# 12. 모델 및 파이프라인 저장
joblib.dump(model_pipeline, 'ML_pipeline.pkl')
print("최종 파이프라인이 'final_pipeline2.pkl'로 저장되었습니다.")


테스트 데이터 정확도: 0.8129
최종 파이프라인이 'final_pipeline2.pkl'로 저장되었습니다.
